In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [2]:
data = pd.read_csv("Churn_Modelling.csv")

In [3]:
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [4]:
#Encode Catgeorical Variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [6]:
onehot_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out())

In [8]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)


In [9]:
import pickle
with open('label_encode_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [10]:
#split the data in dependent and independent features
x=data.drop('Exited',axis=1)
y=data['Exited']

#split the data in training and testing
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)



In [11]:
#scale the training data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [12]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime



In [ ]:
#Build our ANN model
model = Sequential([
    Dense(64,activation = 'relu',input_shape=(12,)), #HL1 connected to the input layer
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [ ]:
#compile the model
model.compile(optimizer = opt , loss = loss, metrics = ['accuracy'])

In [ ]:
#set up TensorBoard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = 'log/fit' 
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
# set up the Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
#training the model
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,callbacks=[tensorboard_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8290 - loss: 0.3999 - val_accuracy: 0.8465 - val_loss: 0.3656
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - accuracy: 0.8537 - loss: 0.3558 - val_accuracy: 0.8505 - val_loss: 0.3532
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 0.8587 - loss: 0.3492 - val_accuracy: 0.8520 - val_loss: 0.3471
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.8618 - loss: 0.3428 - val_accuracy: 0.8605 - val_loss: 0.3410
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.8615 - loss: 0.3386 - val_accuracy: 0.8600 - val_loss: 0.3414
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.8601 - loss: 0.3402 - val_accuracy: 0.8595 - val_loss: 0.3528
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - accuracy: 0.8605 - loss: 0.3361 - val_accuracy: 0.8615 - val_loss: 0.3486
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - accuracy: 0.8626 - loss: 0.3

In [ ]:
model.save('model.h5')

In [ ]:
#load the tensorboard
%load_ext tensorboard

In [23]:
%tensorboard --logdir log/fit